In [10]:
import os
import mysql.connector
from astropy.io import fits

#database config 
db_config = {
    'host':'localhost',
    'user': 'root',
    'password': '', 
    'database':'test', 
}

fits_directory = 'C:\\Users\\samds\\Desktop\\OBS\\nas-frontend'


#stablish connection

conn = mysql.connector.connect(**db_config)
cursor = conn.cursor()



for filename in os.listdir(fits_directory):
    
    if filename.endswith('.fits'):
        file_path = os.path.join(fits_directory, filename)
        
        
        with fits.open(file_path) as fits_file:
            
            header = fits_file[0].header
            
           
            instrument_name = header.get('INSTRUME', 'N/A')
            filter_name = header.get('FILTER', 'N/A')
            date_avg = header.get('DATE-AVG', 'N/A')
            
            #insert the data
cursor.execute('''
    INSERT INTO fits_metadata (file_name, instrument_name, filter, date_avg)
    VALUES (%s, %s, %s, %s)
''', (filename, instrument_name, filter_name, date_avg))

# Print confirmation 
print(f"Inserted data for file: {filename}")



Inserted data for file: test_files.ipynb
